In [22]:
import dgl 
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

import sys
sys.path.append('/Users/tanyanaheta/desktop/nyu/y2s1/1006Capstone/NYU-Zillow-Capstone-2022-Team-A')
import src.datamodules.SAGE as g_train

In [23]:
# Create the graph
dataset = dgl.data.CSVDataset('./graph_csv')
g = dataset[0]

Done loading data from cached files.


In [24]:
######################
## Data Preperation Positive Edges 
######################

test_frac = 0.1

# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)



test_size = int(len(eids) * test_frac)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

print(test_pos_u)

print("Total number of edges: ", len(eids))
print("Total number of training edges: ", train_size)
print("Total number of testing edges: ", test_size)


tensor([1485, 4883, 2639,  ..., 2524, 2654, 1263])
Total number of edges:  14631
Total number of training edges:  13168
Total number of testing edges:  1463


In [25]:
# shortened_adj_matrix = g.adjacency_matrix().clone().to_dense()
# print(shortened_adj_matrix.size())
# print(shortened_adj_matrix)
# tag_loc = torch.ByteTensor(tag_mask)
# print(tag_loc.size())
# print(tag_loc)
# shortened_adj_matrix = torch.transpose(torch.transpose(shortened_adj_matrix,0,1)[tag_loc], 0,1)
# print(shortened_adj_matrix.size())


# tag_mask = list(([1] * 80) + ([0] * 5000))
# #print(tag_mask)
# tag_loc = torch.ByteTensor(tag_mask)
# print(tag_loc.size())

In [26]:
######################
# ## Negative Edges
######################

# Create all negative edges for this graph
neg_edges = dgl.sampling.global_uniform_negative_sampling(g, 100000000)

# First mask used to eliminate all tag nodes as source
mask = neg_edges[0].lt(5000)
print(mask)

# Apply the mask to both source and destination nodes
masked_neg = (torch.masked_select(neg_edges[0], mask), torch.masked_select(neg_edges[1], mask))

# Second mask used to eliminate all image nodes as destination
mask2 = masked_neg[1].ge(5000)
print(mask2)

# Apply the mask to both source and destination nodes
masked_neg2 = (torch.masked_select(masked_neg[0], mask2), torch.masked_select(masked_neg[1], mask2))

# Create a list of remaining source and destination nodes
neg_u = (masked_neg2[0].tolist())
neg_v = (masked_neg2[1].tolist())

tensor([ True,  True,  True,  ..., False, False, False])
tensor([False, False, False,  ...,  True,  True,  True])


In [28]:
# Get all node features that can be imported to the graphs
df = pd.read_csv('/Users/tanyanaheta/desktop/nyu/y2s1/1006Capstone/NYU-Zillow-Capstone-2022-Team-A/graph/graph_csv/coco_nodes.csv')
df['featNew'] = df['feat'].apply(lambda x: [float(y) for y in x.split(",")])
features = torch.Tensor(df['featNew'])

In [31]:
###########################
## Create Training Graph ##
###########################

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get the number of negative edges and split into train, test
neg_eids_len = len(neg_u)
test_size = int((neg_eids_len) * test_frac)
train_size = neg_eids_len - test_size

# Create source and destination nodes list for positive and negative graphs
test_neg_u, test_neg_v = neg_u[:test_size], neg_v[:test_size]
train_neg_u, train_neg_v = neg_u[test_size:], neg_v[test_size:]

train_pos_g = dgl.graph((train_pos_u, train_pos_v))
train_pos_g.ndata['feat'] = features
train_pos_g = train_pos_g.to(device)
test_pos_g = dgl.graph((test_pos_u, test_pos_v)).to(device)
test_pos_g.ndata['feat'] = features[0:5078]
test_pos_g = test_pos_g.to(device)

train_neg_g = dgl.graph((train_neg_u, train_neg_v)).to(device)
train_neg_g.ndata['feat'] = features
train_neg_g = train_neg_g.to(device)
test_neg_g = dgl.graph((test_neg_u, test_neg_v)).to(device)
test_neg_g.ndata['feat'] = features[0:5009]
test_neg_g = test_neg_g.to(device)


In [32]:
########################
## Predicton Function ##
########################

import dgl.function as fn

class DotPredictor(torch.nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g = g.to(device)
            g.ndata['h'] = h.to(device)
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
# WIP
class MLPPredictor(torch.nn.Module):
    # def __init__(self, g):
    #     super().__init__()
    #     self.W1 = torch.nn.Linear(g.ndata['feat'] * 2, g.ndata['feat'])
    #     self.W2 = torch.nn.Linear(g.ndata['feat'], 1)

    # def apply_edges(self, edges):
    #     """
    #     Computes a scalar score for each edge of the given graph.

    #     Parameters
    #     ----------
    #     edges :
    #         Has three members ``src``, ``dst`` and ``data``, each of
    #         which is a dictionary representing the features of the
    #         source nodes, the destination nodes, and the edges
    #         themselves.

    #     Returns
    #     -------
    #     dict
    #         A dictionary of new edge features.
    #     """
    #     # clone_neg = test_neg_g 
    #     # h = torch.cat([edges.src['h'], edges.dst['h']], 1)
    #     # return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}



    #     src_feat = []
    #     dst_feat = []

    #     # Feature array for the first node in the list of edges sourcenodes 
    #     g.ndata['feat'][g.all_edges()[0][0].item()]


    #         # All the node features
    #     # g.ndata['feat']
    #         # Getting the source nodeID 
    #     #g.all_edges()[0][0].item()

    #     #What we want 
    #     torch.cat(['src_feat', 'dst_feat'], 1)

    # def forward(self, g, h):
    #     with g.local_scope():
    #         g.ndata['h'] = h
    #         g.apply_edges(self.apply_edges)
    #         return g.edata['score']

In [33]:
###########################
## Training Loop - Setup ##
###########################

import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

model = g_train.SAGE(train_pos_g.ndata['feat'].shape[1], None ,train_pos_g.ndata['feat'].shape[1], 'mean')
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

## Note: loss can be greater than one because labels are 1s and 0s 
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
#############################
## Training Loop Execution ##
#############################

import itertools
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.05)

for e in range(300):
    # forward
    h = model(train_pos_g, train_pos_g.ndata['feat']).to(device)
    # print(type(h))
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 100 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

In [ ]:
################
## Evaluation ##
################

with torch.no_grad():
    pos_score = pred(test_pos_g.to(device), test_pos_g.ndata['feat'].to(device))
    neg_score = pred(test_neg_g.to(device), test_neg_g.ndata['feat'].to(device))
    print('AUC', compute_auc(pos_score, neg_score))

AUC 0.9861590617804803
